In [26]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [27]:
# How many dishes are available and in what range of calories
DISH_COUNT = 3000
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1500

# User config - how many days, how many dishes per day, how many calories per day
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3
CALORIES_PER_DAY = 2000
# Penalty for dishes repaeating during the week
PENALTY = 400

# Genetic algorithm config
POPULATION_SIZE = 200
MATING_POPULATION_SIZE = 50
ITERATIONS = 1000

#### Helper functions

In [28]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Real data

In [29]:
dishes = {}
data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

while len(dishes) != DISH_COUNT:
    random_choice = df.sample()
    if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
        dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
dishes = list(dishes.items())
print(*dishes[:DAYS_PER_MENU], sep='\n')

('Steak And Cheese Sandwich Plain On Roll', 421.6)
('Frankfurter Or Hot Dog Sandwich Reduced Fat Or Light Plain On Whole Wheat Bun', 193.8)
('Haddock Coated Baked Or Broiled Fat Added In Cooking', 316.2)
('Enchilada With Meat Red-Chile Or Enchilada Sauce', 179.34)
('Cookies Peanut Butter Commercially Prepared Soft-Type', 129.78799999999998)
('Dry Roasted Macadamia Nuts', 947.76)
('Lamb Australian Imported Fresh Rack Roast Frenched Denuded Bone-In Separable Lean And Fat Trimmed To 0 Inch Fat Cooked Roasted', 164.05)


#### Sample random menu

In [30]:
def create_random_menu():
    menu = np.ndarray((DAYS_PER_MENU, DISHES_PER_DAY), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [31]:
def count_calories_per_day(menu):

    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    
    return calories

def count_duplicates_per_day(menu):
    
    names = np.reshape(menu, DAYS_PER_MENU * DISHES_PER_DAY)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1 )
    
    return attr

def score_menu(menu):
    
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [(attr*PENALTY) ** 2 for attr in count_duplicates_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    
    return sum(squares + penalty)

In [32]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 7966764.277852001
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                             │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    571.284 │ [('Beef Chuck Blade Roast Separable Lean And Fat Trimmed To 1/8 Inch Fat Select Cooked Braised', 270.3)            │
│       │            │  ('Dark Chocolate (45-59% Cocoa)', 155.064)                                                                        │
│       │            │  ('Noodles Egg Dry Enriched', 145.92)]                                                                             │
├───────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [33]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [34]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [35]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [36]:
population.sort(key=score_menu)
visualize_menu(population[0])

Score: 2.8335999999998025
╒═══════╤════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                │
╞═══════╪════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    2000.19 │ [('Submarine Sandwich Steak And Cheese On White Bread With Cheese Lettuce And Tomato', 367.83)        │
│       │            │  ('Seeds Sunflower Seed Kernels Dry Roasted With Salt Added', 744.96)                                 │
│       │            │  ('Confectioners Coating Yogurt', 887.4)]                                                             │
├───────┼────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    1999.67 │ [('Corn Flour Whole-Grain White', 422.37) ('Barley Pearled Raw